<a href="https://colab.research.google.com/github/samiemostafavi/ros-latency-pr3d/blob/main/ros_cross_traffic_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('bryan-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

password = getpass('enter your expeca password:')
os.environ['OS_PASSWORD'] = password

enter your expeca password:··········


Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.6/341.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.0/207.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.9/517.9 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

Import packages

In [ ]:
import json
from loguru import logger
import chi.network, chi.container
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, show_reservation_byname, get_available_publicips, get_segment_ids, get_worker_interfaces

In the next cell, we reserve the required equipment and resources to form an end to end experiment setup. We reserve EP5G network.

In [ ]:
# ep5g reservation
ep5g_lease = show_reservation_byname("ep5g-lease")
segment_ids = get_segment_ids('ep5g')
if not ep5g_lease:
    ep5g_lease = reserve(
        { "type":"network", "name": "ep5g", "net_name": "ep5g-vip", "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )


leaseslist = list_reservations(brief=True)
print(json.dumps(leaseslist,indent=4))

2025-01-30 15:26:48.216 | INFO     | chi.expeca:reserve:243 - reserving ep5g
2025-01-30 15:26:50.818 | INFO     | chi.expeca:wait_until_lease_status:138 - waiting 120 seconds for ep5g-lease with id 148d3f5e-39cf-4d2a-902c-d5a71f682fad to become "ACTIVE"
2025-01-30 15:26:55.978 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 148d3f5e-39cf-4d2a-902c-d5a71f682fad is PENDING.
2025-01-30 15:27:01.130 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 148d3f5e-39cf-4d2a-902c-d5a71f682fad is PENDING.
2025-01-30 15:27:06.286 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 148d3f5e-39cf-4d2a-902c-d5a71f682fad is STARTING.
2025-01-30 15:27:11.446 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 148d3f5e-39cf-4d2a-902c-d5a71f682fad is STARTING.
2025-01-30 15:27:16.593 | INFO     | chi.expeca:wait_until_lease_status:145 - lease ep5g-lease with id 148d3f5e-39cf-4d2a-902c-d5a71f682

[
    {
        "name": "ep5g-lease",
        "id": "2bb82db1-bd83-4285-b6a6-bedceef24c35",
        "reservation_id": "e79859ae-31fd-4b3e-b647-542e2f0ef879",
        "status": "ACTIVE",
        "end_date": "2025-02-06T15:26:00.000000"
    }
]


In the following section we setup the networking equipment for ep5g, edge-net, Telenor Edge, and Internet (refer to [here](https://kth-expeca.gitbook.io/testbedconfig/enroll/enroll-network-segments/ep5g) for more info).

In [ ]:
# create edge-net
edgenet = chi.network.create_network("edge-net")
chi.network.create_subnet("edge-net-subnet",edgenet["id"],"10.70.70.0/24",gateway_ip=None,enable_dhcp=False)
logger.success("edge-net is created.")

# create ep5g-vip-router
router = chi.network.create_router("ep5g-vip-router", "public")
logger.success("ep5g-vip-router router is created.")
logger.info(f"{json.dumps(router,indent=4)}")

# connect ep5g-vip-net to ep5g-vip-router
ep5gnet = chi.network.get_network("ep5g-vip-net")
portadd = chi.network.add_subnet_to_router(router["id"],ep5gnet["subnets"][0])
logger.success("An interface on ep5g-vip-net is added to the router")

# connect edge-net to ep5g-vip-router
edgenet = chi.network.get_network("edge-net")
portadd = chi.network.add_subnet_to_router(router["id"],edgenet["subnets"][0])
logger.success("An interface on edge-net is added to the router")

# add ep5g route to ep5g-vip-router
routeadd = chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.30.111.10")
logger.success("the ep5g route added to the router")

2025-01-30 15:28:55.883 | SUCCESS  | __main__:<cell line: 0>:4 - edge-net is created.
2025-01-30 15:28:58.865 | SUCCESS  | __main__:<cell line: 0>:8 - ep5g-vip-router router is created.
2025-01-30 15:28:58.867 | INFO     | __main__:<cell line: 0>:9 - {
    "id": "fe89d065-2ff4-44f7-bea1-f4c0f2d1e478",
    "name": "ep5g-vip-router",
    "tenant_id": "c47f3ed65941413fbc6087028aff5ee2",
    "admin_state_up": true,
    "status": "ACTIVE",
    "external_gateway_info": {
        "network_id": "717b5f2b-069e-4868-a24d-91a4ae3ad002",
        "external_fixed_ips": [
            {
                "subnet_id": "53d03ffd-0d49-4f9a-88a3-a30d69fe4827",
                "ip_address": "130.237.11.109"
            }
        ],
        "enable_snat": true
    },
    "description": "",
    "availability_zones": [],
    "availability_zone_hints": [],
    "routes": [],
    "flavor_id": null,
    "tags": [],
    "created_at": "2025-01-30T15:28:56Z",
    "updated_at": "2025-01-30T15:28:57Z",
    "revision_num

In [ ]:
# worker reservation
worker_name = "worker-06"
worker_lease = show_reservation_byname(worker_name + "-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

# worker reservation
worker_name = "worker-07"
worker_lease = show_reservation_byname(worker_name + "-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

# worker reservation
worker_name = "worker-08"
worker_lease = show_reservation_byname(worker_name + "-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

# worker reservation
worker_name = "worker-09"
worker_lease = show_reservation_byname(worker_name + "-lease")
if not worker_lease:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":7, "hours":0 } }
    )
worker_reservation_id = worker_lease["reservations"][0]["id"]

2025-01-30 15:30:43.780 | INFO     | chi.expeca:reserve:243 - reserving worker-06
2025-01-30 15:30:46.067 | INFO     | chi.expeca:wait_until_lease_status:138 - waiting 120 seconds for worker-06-lease with id acc133c3-25e0-43ca-aeba-82787520cbb5 to become "ACTIVE"
2025-01-30 15:30:51.209 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-06-lease with id acc133c3-25e0-43ca-aeba-82787520cbb5 is PENDING.
2025-01-30 15:30:56.373 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-06-lease with id acc133c3-25e0-43ca-aeba-82787520cbb5 is PENDING.
2025-01-30 15:31:01.532 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-06-lease with id acc133c3-25e0-43ca-aeba-82787520cbb5 is PENDING.
2025-01-30 15:31:06.686 | INFO     | chi.expeca:wait_until_lease_status:145 - lease worker-06-lease with id acc133c3-25e0-43ca-aeba-82787520cbb5 is ACTIVE.
2025-01-30 15:31:06.688 | SUCCESS  | chi.expeca:try_to_create_lease:285 - done
2025-01-30 15:31:06.856 | INFO

In [ ]:
# advantech router reservation
adv_name = "adv-02"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# advantech router reservation
adv_name = "adv-03"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# advantech router reservation
adv_name = "adv-04"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# advantech router reservation
adv_name = "adv-05"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# advantech router reservation
adv_name = "adv-06"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

# advantech router reservation
adv_name = "adv-07"
segment_ids = get_segment_ids(adv_name)
adv_lease = show_reservation_byname(adv_name + "-lease")
if not adv_lease:
    adv_lease = reserve(
        { "type":"network", "name": adv_name, "net_name": adv_name, "segment_id": segment_ids['rj45'], "duration": { "days":7, "hours":0 } }
    )

2025-01-30 15:46:38.971 | INFO     | chi.expeca:reserve:243 - reserving adv-05
2025-01-30 15:46:41.289 | INFO     | chi.expeca:wait_until_lease_status:138 - waiting 120 seconds for adv-05-lease with id 06811893-f40e-4ceb-9098-3c1ee86c005f to become "ACTIVE"
2025-01-30 15:46:46.573 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-05-lease with id 06811893-f40e-4ceb-9098-3c1ee86c005f is PENDING.
2025-01-30 15:46:51.846 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-05-lease with id 06811893-f40e-4ceb-9098-3c1ee86c005f is PENDING.
2025-01-30 15:46:57.123 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-05-lease with id 06811893-f40e-4ceb-9098-3c1ee86c005f is PENDING.
2025-01-30 15:47:02.499 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-05-lease with id 06811893-f40e-4ceb-9098-3c1ee86c005f is PENDING.
2025-01-30 15:47:07.784 | INFO     | chi.expeca:wait_until_lease_status:145 - lease adv-05-lease with id 06811893-f40e-4ceb-90

In [ ]:
# worker reservation
worker_name = "worker-07"
worker_lease = show_reservation_byname(worker_name + "-lease")
worker_reservation_id = worker_lease["reservations"][0]["id"]
edgenet = chi.network.get_network("edge-net")
container_name = "ct-server-node"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    mounts = [],
    nets = [
        { "network" : edgenet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":"10.70.70.210/24",
        "networks.1.routes":"172.16.0.0/16-10.70.70.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-01-30 16:08:54.040 | SUCCESS  | __main__:<cell line: 0>:23 - created ct-server-node container.


In [ ]:
ports = [
    53301,
    53302,
    53303,
    53304,
    53305
]
container_name = "ct-server-node"
for port in ports:
    command = f"iperf3 -s -p {port}"
    result = chi.container.execute(
        container_ref=container_name,
        command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
    )
    logger.info(f"{result}")

2025-01-30 16:09:25.643 | INFO     | __main__:<cell line: 0>:15 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
2025-01-30 16:09:25.985 | INFO     | __main__:<cell line: 0>:15 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
2025-01-30 16:09:26.337 | INFO     | __main__:<cell line: 0>:15 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
2025-01-30 16:09:26.696 | INFO     | __main__:<cell line: 0>:15 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}
2025-01-30 16:09:27.046 | INFO     | __main__:<cell line: 0>:15 - {'output': 'Command received and started in the background.\n', 'exit_code': 0, 'exec_id': None, 'proxy_url': None}


In [ ]:
advnet = chi.network.get_network("adv-02-net")
container_name = "ct-client-node-01"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12409",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-01-30 16:10:56.314 | SUCCESS  | __main__:<cell line: 0>:18 - created ct-client-node-01 container.


In [ ]:
advnet = chi.network.get_network("adv-03-net")
container_name = "ct-client-node-02"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12419",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-01-30 16:11:45.042 | SUCCESS  | __main__:<cell line: 0>:18 - created ct-client-node-02 container.


In [ ]:
advnet = chi.network.get_network("adv-04-net")
container_name = "ct-client-node-03"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12429",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-01-30 16:12:29.182 | SUCCESS  | __main__:<cell line: 0>:18 - created ct-client-node-03 container.


In [ ]:
worker_name = "worker-08"
worker_lease = show_reservation_byname(worker_name + "-lease")
worker_reservation_id = worker_lease["reservations"][0]["id"]

advnet = chi.network.get_network("adv-05-net")
container_name = "ct-client-node-04"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-01-30 16:13:31.579 | SUCCESS  | __main__:<cell line: 0>:22 - created ct-client-node-04 container.


In [ ]:
advnet = chi.network.get_network("adv-06-net")
container_name = "ct-client-node-05"
chi.container.create_container(
    name = container_name,
    image = "samiemostafavi/perf-meas",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"eno12409",
        "networks.1.ip":"10.42.3.2/24",
        "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-01-30 16:14:23.508 | SUCCESS  | __main__:<cell line: 0>:18 - created ct-client-node-05 container.


In [ ]:
worker_name = "worker-06"
worker_lease = show_reservation_byname(worker_name + "-lease")
worker_reservation_id = worker_lease["reservations"][0]["id"]

publicnet = chi.network.get_network("serverpublic")
edgenet = chi.network.get_network("edge-net")
container_name = "ros-latency-server-node"
chi.container.create_container(
    name = container_name,
    image = "bryandony/ros:melodic-voxblox-edge-expeca",
    reservation_id = worker_reservation_id,
    environment = {
        "DNS_IP":"8.8.8.8",
        "GATEWAY_IP":"130.237.11.97",
        "PASS":"expeca"
    },
    mounts = [],
    nets = [
        { "network" : publicnet['id'] },
        { "network" : edgenet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1f1",
        "networks.1.ip":"130.237.11.122/27",
        "networks.1.gateway":"130.237.11.97",
        "networks.2.interface":"eno12399np0",
        "networks.2.ip":"10.70.70.211/24",
        "networks.2.routes":"172.16.0.0/16-10.70.70.1",
        "capabilities.privileged":"true",
    },
)
chi.container.wait_for_active(container_name)
logger.success(f"created {container_name} container.")

2025-01-30 16:16:33.092 | SUCCESS  | __main__:<cell line: 0>:33 - created ros-latency-server-node container.


In [ ]:
worker_name = "worker-09"
worker_lease = show_reservation_byname(worker_name + "-lease")
worker_reservation_id = worker_lease["reservations"][0]["id"]
publicnet = chi.network.get_network("serverpublic")
advnet = chi.network.get_network("adv-07-net")
name = "ros-latency-client-node"
chi.container.create_container(
    name = name,
    image = "bryandony/ros:melodic-voxblox-edge-expeca",
    reservation_id = worker_reservation_id,
    environment = {"SERVER_DIR":"/tmp/"},
    nets = [
        { "network" : publicnet['id'] },
        { "network" : advnet['id'] },
    ],
    labels = {
        "networks.1.interface":"ens1",
        "networks.1.ip":"130.237.11.123/27",
        "networks.1.gateway":"130.237.11.97",
        "networks.2.interface":"eno12409",
        "networks.2.ip":"10.42.3.2/24",
        "networks.2.routes":"10.70.70.0/24-10.42.3.1",
        "capabilities.privileged":"true",
    },
)
chi.container.wait_for_active(name)
logger.success(f"created {name} container.")

2025-01-30 16:22:08.861 | SUCCESS  | __main__:<cell line: 0>:27 - created ros-latency-client-node container.
